## Installing fastText

In [ ]:
from src.fast_word import *

--2021-10-06 22:10:31--  https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
Résolution de github.com (github.com)… 140.82.121.3
Connexion à github.com (github.com)|140.82.121.3|:443… connecté.
requête HTTP transmise, en attente de la réponse… 302 Found
Emplacement : https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2 [suivant]
--2021-10-06 22:10:32--  https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2
Résolution de codeload.github.com (codeload.github.com)… 140.82.121.9
Connexion à codeload.github.com (codeload.github.com)|140.82.121.9|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : non indiqué [application/zip]
Enregistre : ‘v0.9.2.zip’

v0.9.2.zip              [    <=>             ]   4,17M  6,39MB/s    ds 0,7s    

2021-10-06 22:10:32 (6,39 MB/s) - ‘v0.9.2.zip’ enregistré [4369852]

Archive:  v0.9.2.zip
5b5943c118b0ec5fb9cd8d20587de2b2d3966dfe
   creating: fastText-0.9.2/
   creating: fastText-0.9.2/.circleci/

  inflating: fastText-0.9.2/website/static/docs/en/html/search/variables_3.html  
  inflating: fastText-0.9.2/website/static/docs/en/html/search/variables_3.js  
  inflating: fastText-0.9.2/website/static/docs/en/html/search/variables_4.html  
  inflating: fastText-0.9.2/website/static/docs/en/html/search/variables_4.js  
  inflating: fastText-0.9.2/website/static/docs/en/html/search/variables_5.html  
  inflating: fastText-0.9.2/website/static/docs/en/html/search/variables_5.js  
  inflating: fastText-0.9.2/website/static/docs/en/html/search/variables_6.html  
  inflating: fastText-0.9.2/website/static/docs/en/html/search/variables_6.js  
  inflating: fastText-0.9.2/website/static/docs/en/html/search/variables_7.html  
  inflating: fastText-0.9.2/website/static/docs/en/html/search/variables_7.js  
  inflating: fastText-0.9.2/website/static/docs/en/html/search/variables_8.html  
  inflating: fastText-0.9.2/website/static/docs/en/html/search/variables_8.js  
  inflating: fastText-0.9.2/

  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-linux_x86_64.whl size=4168004 sha256=acf00d7aa4a61bb4a3e39222d0a2d5a25382a48cfc7cc7fc65ba619c085e4c99
  Stored in directory: /home/token/.cache/pip/wheels/05/02/87/cedb84c3bafd59b4d41ab1fa00207f56e80b5137c88cb55d7e
Successfully built fasttext
  Attempting uninstall: fasttext
    Found existing installation: fasttext 0.9.2
    Uninstalling fasttext-0.9.2:
      Successfully uninstalled fasttext-0.9.2


The first step of this tutorial is to install and build fastText.

### Download directly with command line the english dataset

In [6]:
!cd fastText-0.9.2 && python3 download_model.py en

File exists. Use --overwrite to download anyway.


### Loading the dataset "IMDB"

Using the split argument, we can split the imdb into two separate dataset.

In [124]:
train_dataset = load_dataset('imdb', split='train')
test_dataset = load_dataset('imdb', split='test')

Reusing dataset imdb (/home/token/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)
Reusing dataset imdb (/home/token/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


*train_dataset* is a class with two attributes :
- Features which contains two features : text and label (our x_train and our y_train)
- The number of rows in our dataset

In [125]:
print(train_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


If we take the first sample of our training dataset, we can see the first review and the label (positive or negative) according to that review.

In [126]:
print(train_dataset[0])

{'text': 'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!', 'label': 1}


### Preprocessing the dataset

With the review we previously saw, the review are definitely not preprocessed. There is still some html tags etc.. 
We definitely to work on those.

In [127]:
y_train = train_dataset['label']

In [128]:
y_test = test_dataset['label']

In [129]:
create_imdb_train_file(train_dataset, y_train)
create_imdb_train_and_validation_file(train_dataset, y_train)
create_imdb_test_file(test_dataset, y_test)

100%|██████████| 25000/25000 [00:24<00:00, 1032.41it/s]


## Let's train the model using fasttext.

##### 1) Lemming

In [130]:
create_imdb_train_file(train_dataset, y_train, lemm = True)
create_imdb_train_and_validation_file(train_dataset, y_train, lemm = True)
create_imdb_test_file(test_dataset, y_test, lemm = True)

100%|██████████| 25000/25000 [00:55<00:00, 453.75it/s]


In [131]:
%%time
model = fasttext.train_supervised(input="imdb_train_lemmed.txt")

CPU times: user 21.3 s, sys: 520 ms, total: 21.8 s
Wall time: 4.92 s


In [132]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.86904, Rappel: 0.86904


##### 2) Stemming

In [133]:
create_imdb_train_file(train_dataset, y_train, stemm = True)
create_imdb_train_and_validation_file(train_dataset, y_train, stemm = True)
create_imdb_test_file(test_dataset, y_test, stemm = True)

100%|██████████| 25000/25000 [01:54<00:00, 217.94it/s]


In [134]:
%%time
model = fasttext.train_supervised(input="imdb_train_stemmed.txt")

CPU times: user 22.1 s, sys: 238 ms, total: 22.4 s
Wall time: 4.45 s


In [135]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.80024, Rappel: 0.80024


##### 3) Stop Words

In [136]:
create_imdb_train_file(train_dataset, y_train, stop_words = True)
create_imdb_train_and_validation_file(train_dataset, y_train, stop_words = True)
create_imdb_test_file(test_dataset, y_test, stop_words = True)

100%|██████████| 25000/25000 [00:18<00:00, 1381.93it/s]


In [137]:
%%time
model = fasttext.train_supervised(input="imdb_train_sw.txt")

CPU times: user 10.9 s, sys: 167 ms, total: 11.1 s
Wall time: 2.22 s


In [138]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.87424, Rappel: 0.87424


##### 4) None

In [139]:
%%time
model = fasttext.train_supervised(input="imdb_train.txt")

CPU times: user 21.2 s, sys: 243 ms, total: 21.4 s
Wall time: 3.9 s


In [140]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.8758, Rappel: 0.8758


### Making the model better

#### How about N-grams ?

In [141]:
%%time
model = fasttext.train_supervised(input="imdb_train.txt", wordNgrams=2) 

CPU times: user 48.6 s, sys: 837 ms, total: 49.4 s
Wall time: 9.42 s


In [142]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.87156, Rappel: 0.87156


#### With more epochs and a smaller learning_rate ?

In [143]:
%%time
model = fasttext.train_supervised(input="imdb_train.txt", lr=0.5, epoch=25, wordNgrams=2)

CPU times: user 3min 37s, sys: 1.61 s, total: 3min 38s
Wall time: 34.5 s


In [144]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.89668, Rappel: 0.89668


#### With more epochs and a smaller smaller learning_rate ?

In [145]:
%%time
model = fasttext.train_supervised(input="imdb_train.txt", lr=0.25, epoch=25, wordNgrams=2)

CPU times: user 3min 44s, sys: 1.82 s, total: 3min 45s
Wall time: 35 s


In [146]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.89672, Rappel: 0.89672


#### With more epochs and a bigger learning_rate ?

In [147]:
%%time
model = fasttext.train_supervised(input="imdb_train.txt", lr=1.0, epoch=25, wordNgrams=2)

CPU times: user 3min 43s, sys: 1.81 s, total: 3min 45s
Wall time: 34.7 s


In [148]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.89532, Rappel: 0.89532


#### With the autotuneValidationfile (testing and performance purpose)

In [149]:
%%time
model = fasttext.train_supervised(input="imdb_train_splited_with_the_validation.txt", autotuneValidationFile='imdb_validation.txt', autotuneDuration=150)

CPU times: user 17min 21s, sys: 7.12 s, total: 17min 28s
Wall time: 2min 51s


In [150]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.854, Rappel: 0.854


## Beating the baseline

### Glove

Special preprocessing for glove.

In [161]:
x_train = []
x_test = []

for i in range(train_dataset.num_rows):
    x_train.append(pre_process_document(train_dataset[i]['text'], False, False, False))
    

for i in range(test_dataset.num_rows):
    x_test.append(pre_process_document(test_dataset[i]['text'], False, False, False))

EmbeddingTransformer with glove

In [156]:
glove = EmbeddingTransformer('glove')
x_train = glove.transform(x_train)

Call the logistic regression and fit it with the x_train reshaped and transformed.

In [157]:
model = LogisticRegression(random_state=42, solver='lbfgs', max_iter=1000)
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

Predict the testing dataset that we have and let's see how well we performed compared to our baseline.

In [162]:
x_test = glove.transform(x_test)
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.77      0.76     12500
           1       0.76      0.73      0.75     12500

    accuracy                           0.75     25000
   macro avg       0.75      0.75      0.75     25000
weighted avg       0.75      0.75      0.75     25000

